In [1]:
import os

# get absolute path
base_path = os.getcwd()
print(base_path)

#get data syllable path
data_path = os.path.join(base_path, 'data\\syllable-level')
print(data_path)

d:\IDE_workspace\VSCode\TLCN_Project\ViText2SQL
d:\IDE_workspace\VSCode\TLCN_Project\ViText2SQL\data\syllable-level


In [2]:
train_dataset_path = os.path.join(data_path, 'train.json')
table_dataset_path = os.path.join(data_path, 'tables.json')

In [3]:
import json

train_dataset = json.load(open(train_dataset_path))
table_dataset = json.load(open(table_dataset_path))

In [7]:
def get_table_units(sql):
    lst_id = set()
    for item in sql['from']["table_units"]:
        if item[0] == "table_unit":
            lst_id.add(item[1])
        elif item[0] == "sql":
            lst_id.update(get_table_units(item[1]))
    if sql["except"]:
        lst_id.update(get_table_units(sql["except"]))
    if sql["union"]:
        lst_id.update(get_table_units(sql["union"]))
    if sql["intersect"]:
        lst_id.update(get_table_units(sql["intersect"]))
    return lst_id

def see_query_info(row):
    sql = row['sql']
    table_units = get_table_units(sql)

    return row['question'], row['query'], table_units

def see_table_schema(schema):
    tables = [(r, []) for r in schema['table_names']]
    for col in schema['column_names'][1:]:
        # add column to coresponding table
        tables[col[0]][1].append(col[1])
    return tables

def find_db_from_query(data):
    db_id = data['db_id']
    for table in table_dataset:
        if table['db_id'] == db_id:
            return table

def find_all_question_and_query_from_table(data):
    db_id = data['db_id']
    question_n_query = []
    for r in train_dataset:
        if r['db_id'] == db_id:
            question_n_query.append(see_query_info)
    return question_n_query

In [8]:
def all_info(data):
    tables = find_db_from_query(data)
    question, query, table_units = see_query_info(data)
    schema = see_table_schema(tables)
    try:
        core_respond_table = [schema[unit] for unit in table_units]
        return core_respond_table, question, query, schema
    except:
        print(query)
    

In [10]:
id = 10
core_respond_table, query, question, schema = all_info(train_dataset[id])
display(core_respond_table)
print(query)
print(question)

[('tổ chức', ['lục địa', 'trang chủ', 'tên', 'id tổ chức'])]

cho biết trang chủ của ' Đại học Michigan ' .
select trang chủ from tổ chức where tên = "University of Michigan"


In [11]:
def table_to_string(tables):
    lst = []
    for table in tables:
        lst.append(table[0] + ' (' + ','.join(table[1]) + ')')
    result = '; '.join(lst)
    return result    

def to_dataset(data):
    core_respond_table, question, query, _ = all_info(data)
    
    # ouput is dataset with inputs and lables
    columns = ["inputs", "labels"]
    data_single_input = {}
    instruction = f"Tôi có bảng: {table_to_string(core_respond_table)}. Hãy trả lời bằng câu truy vấn: {question}"

    data_single_input[columns[0]] = instruction
    data_single_input[columns[1]] = query

    #output is dataset with schema, question, query
    columns = ["schema", "question", "labels"]
    data_multiple_input = {}
    data_multiple_input[columns[0]] = table_to_string(core_respond_table)
    data_multiple_input[columns[1]] = question
    data_multiple_input[columns[2]] = query

    return data_single_input, data_multiple_input

In [12]:
to_dataset(train_dataset[2])

({'inputs': "Tôi có bảng: bài báo (tóm tắt,id hội nghị,số lượng trích dẫn,id tạp chí,id bài báo,số lượng trích dẫn,tiêu đề,năm). Hãy trả lời bằng câu truy vấn: cho biết bản tóm tắt của bài báo với tiêu đề là ' làm cho hệ thống cơ sở dữ liệu có thể sử dụng được ' .",
  'labels': 'select tóm tắt from bài báo where tiêu đề = "Making database systems usable"'},
 {'schema': 'bài báo (tóm tắt,id hội nghị,số lượng trích dẫn,id tạp chí,id bài báo,số lượng trích dẫn,tiêu đề,năm)',
  'question': "cho biết bản tóm tắt của bài báo với tiêu đề là ' làm cho hệ thống cơ sở dữ liệu có thể sử dụng được ' .",
  'labels': 'select tóm tắt from bài báo where tiêu đề = "Making database systems usable"'})

In [13]:
def generate_dataset(lst_id, data):
    lst_single_input = []
    lst_multiple_input = []
    for id in lst_id:
        single_input, multiple_input = to_dataset(data[id])
        lst_single_input.append(single_input)
        lst_multiple_input.append(multiple_input)
    return lst_single_input, lst_multiple_input

### Generate Specific id

In [14]:
import random
seed = 20110220
num = 100
random.seed(seed)
lst_id = random.sample(range(len(train_dataset)), num)
print(lst_id)

[426, 6007, 212, 2548, 3035, 6477, 2592, 4995, 2081, 825, 2947, 4732, 210, 5993, 3010, 500, 3894, 5679, 5276, 5770, 4781, 2107, 474, 6506, 3835, 3792, 2623, 3320, 4185, 3203, 5505, 3886, 3486, 5526, 2246, 3946, 1198, 6076, 2265, 1303, 4800, 1588, 5051, 2620, 4031, 958, 5126, 5216, 6393, 5243, 3516, 1903, 613, 412, 559, 3435, 350, 2162, 5098, 1431, 4402, 3912, 6274, 3513, 3515, 6378, 914, 1450, 957, 4266, 1802, 4612, 399, 5365, 4604, 3452, 1213, 2301, 6072, 1089, 6727, 5448, 1631, 18, 4064, 343, 6473, 100, 5366, 1241, 6493, 2518, 657, 4066, 2180, 829, 786, 5151, 3559, 6547]


In [15]:
single_input_dataset, multiple_input_dataset = generate_dataset(lst_id, train_dataset)

In [16]:
multiple_input_dataset[:5]

[{'schema': 'cầu thủ (id cầu thủ,năm sinh,tháng sinh,ngày sinh,đất nước nơi sinh,tiểu bang nơi sinh,thành phố nơi sinh,năm mất,tháng mất,ngày mất,đất nước nơi mất,tiểu bang nơi mất,thành phố nơi mất,tên,họ,tên đệm,cân nặng,chiều cao,tay đánh thuận,tay ném thuận,ngày ra mắt,ngày chơi trận cuối,id trong retro,id trong brref)',
  'question': 'Cho biết 3 quốc gia có ít cầu thủ nhất .',
  'labels': 'select đất nước nơi sinh from cầu thủ group by đất nước nơi sinh order by count ( * ) asc limit 3'},
 {'schema': 'khách hàng (id,tên,họ,công ty,địa chỉ,thành phố,tiểu bang,quốc gia,mã bưu điện,số điện thoại,số fax,e-mail,id đại diện hỗ trợ)',
  'question': "Có bao nhiêu khách hàng sống ở ' Prague ' ?",
  'labels': 'select count ( * ) from khách hàng where thành phố = "Prague"'},
 {'schema': 'phi công (id phi công,tên,tuổi tác)',
  'question': 'Sắp xếp tên của tất cả các phi công theo thứ tự giảm dần về độ tuổi .',
  'labels': 'select tên from phi công order by tuổi tác desc'},
 {'schema': 'sân b

In [29]:
json.dump(single_input_dataset, open('generate_data/single_input_dataset.json', 'w'), ensure_ascii=False, separators=(',', ': '), indent=4)
json.dump(multiple_input_dataset, open('generate_data/multiple_input_dataset.json', 'w'), ensure_ascii=False, separators=(',', ': '), indent=4)

### Generate all data

In [17]:
all_lst_id = [*range(len(train_dataset))]
# random.shuffle(all_lst_id)
print(len(all_lst_id) == len(train_dataset))

True


In [18]:
all_single_input_dataset, all_multiple_input_dataset = generate_dataset(all_lst_id, train_dataset)

In [19]:
json.dump(all_single_input_dataset, open('generate_data/all_single_input_dataset.json', 'w'), ensure_ascii=False, separators=(',', ': '), indent=4)
json.dump(all_multiple_input_dataset, open('generate_data/all_multiple_input_dataset.json', 'w'), ensure_ascii=False, separators=(',', ': '), indent=4)

In [22]:
len(all_single_input_dataset)

6831